# Piecewise Effects

Model non-linear investment costs like economies of scale.

This notebook covers:

- **PiecewiseEffects**: Non-linear cost functions for investments
- **Economies of scale**: Larger systems cost less per unit
- **Comparison**: Linear vs piecewise investment costs

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Economies of Scale

Investment costs often **decrease per unit** at larger sizes:

| Battery Size | Cost per kWh | Reason |
|-------------|--------------|--------|
| 0-100 kWh | 0.15 €/kWh | Small: high overhead per unit |
| 100-300 kWh | 0.10 €/kWh | Medium: better economies |
| 300-600 kWh | 0.05 €/kWh | Large: bulk discounts |

!!! note "Simplified Daily-Equivalent Costs"
    These costs represent **daily-equivalent annualized investment costs**, not one-time purchase prices.
    Real battery costs (€200-400/kWh) would be amortized over years of operation.
    For this single-day example, we use simplified values.

With **linear costs** (constant €/kWh), the optimizer chooses based on marginal value.  
With **piecewise costs**, larger sizes become more attractive due to economies of scale.

## Define the Cost Curve

`PiecewiseEffects` maps size ranges to **cumulative** cost ranges:

```
Size:  0 ----100---- 300 -------- 600
Cost:  0 ---15€---- 35€ -------- 50€
          0.15€/kWh   0.10€/kWh    0.05€/kWh
```

The cost values are **cumulative totals**, not per-segment costs.

In [ ]:
# Piecewise costs: larger systems cost less per kWh
piecewise_costs = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise(
        [
            fx.Piece(start=0, end=100),  # Segment 1: 0-100 kWh
            fx.Piece(start=100, end=300),  # Segment 2: 100-300 kWh
            fx.Piece(start=300, end=600),  # Segment 3: 300-600 kWh
        ]
    ),
    piecewise_shares={
        'costs': fx.Piecewise(
            [
                fx.Piece(start=0, end=15),  # 0-100 kWh → 0-15€ (0.15 €/kWh)
                fx.Piece(start=15, end=35),  # 100-300 kWh → 15-35€ (0.10 €/kWh)
                fx.Piece(start=35, end=50),  # 300-600 kWh → 35-50€ (0.05 €/kWh)
            ]
        )
    },
)

### Cost Calculation Examples

The cost curve is **cumulative** - each segment builds on the previous:

| Size | Calculation | Total Cost | Avg €/kWh |
|------|-------------|------------|----------|
| 50 kWh | 50 × 0.15 | 7.50 € | 0.150 |
| 100 kWh | 100 × 0.15 | 15.00 € | 0.150 |
| 200 kWh | 15 + 100×0.10 | 25.00 € | 0.125 |
| 400 kWh | 15 + 20 + 100×0.05 | 40.00 € | 0.100 |
| 600 kWh | 15 + 20 + 15 | 50.00 € | 0.083 |

## Simple Scenario

A battery arbitrages between cheap night electricity and expensive day electricity.

In [ ]:
# One day, hourly resolution
timesteps = pd.date_range('2024-01-01', periods=24, freq='h')

# Electricity price: cheap at night, expensive during day
elec_price = np.array(
    [
        0.05,
        0.05,
        0.05,
        0.05,
        0.05,
        0.05,  # 00-06: night (cheap)
        0.15,
        0.20,
        0.25,
        0.25,
        0.20,
        0.15,  # 06-12: morning
        0.15,
        0.20,
        0.25,
        0.30,
        0.30,
        0.25,  # 12-18: afternoon (expensive)
        0.20,
        0.15,
        0.10,
        0.08,
        0.06,
        0.05,  # 18-24: evening
    ]
)

# Constant demand
demand = np.full(24, 100)  # 100 kW constant

px.line(x=timesteps, y=elec_price, title='Electricity Price Profile', labels={'x': 'Time', 'y': 'Price [€/kWh]'})

## Model 1: Linear Investment Cost

Constant **0.15 €/kWh** regardless of size (same as smallest piecewise segment).

In [ ]:
fs_linear = fx.FlowSystem(timesteps)

fs_linear.add_elements(
    fx.Bus('Elec'),
    fx.Effect('costs', '€', is_standard=True, is_objective=True),
    # Grid with time-varying price
    fx.Source('Grid', outputs=[fx.Flow('Elec', bus='Elec', size=500, effects_per_flow_hour=elec_price)]),
    # Battery with LINEAR investment cost
    fx.Storage(
        'Battery',
        charging=fx.Flow('charge', bus='Elec', size=fx.InvestParameters(maximum_size=300)),
        discharging=fx.Flow('discharge', bus='Elec', size=fx.InvestParameters(maximum_size=300)),
        capacity_in_flow_hours=fx.InvestParameters(
            effects_of_investment_per_size={'costs': 0.15},  # Constant 0.15 €/kWh
            minimum_size=0,
            maximum_size=600,
        ),
        eta_charge=0.95,
        eta_discharge=0.95,
        initial_charge_state=0,
    ),
    # Demand
    fx.Sink('Demand', inputs=[fx.Flow('Elec', bus='Elec', size=1, fixed_relative_profile=demand)]),
)

fs_linear.optimize(fx.solvers.HighsSolver())

## Model 2: Piecewise Investment Cost

Cost per kWh **decreases** at larger sizes (economies of scale).

In [ ]:
fs_piecewise = fx.FlowSystem(timesteps)

fs_piecewise.add_elements(
    fx.Bus('Elec'),
    fx.Effect('costs', '€', is_standard=True, is_objective=True),
    # Grid with time-varying price
    fx.Source('Grid', outputs=[fx.Flow('Elec', bus='Elec', size=500, effects_per_flow_hour=elec_price)]),
    # Battery with PIECEWISE investment cost
    fx.Storage(
        'Battery',
        charging=fx.Flow('charge', bus='Elec', size=fx.InvestParameters(maximum_size=300)),
        discharging=fx.Flow('discharge', bus='Elec', size=fx.InvestParameters(maximum_size=300)),
        capacity_in_flow_hours=fx.InvestParameters(
            piecewise_effects_of_investment=piecewise_costs,  # Economies of scale
            minimum_size=0,
            maximum_size=600,
        ),
        eta_charge=0.95,
        eta_discharge=0.95,
        initial_charge_state=0,
    ),
    # Demand
    fx.Sink('Demand', inputs=[fx.Flow('Elec', bus='Elec', size=1, fixed_relative_profile=demand)]),
)

fs_piecewise.optimize(fx.solvers.HighsSolver())

## Visualize the Cost Curve

In [ ]:
fs_piecewise.components['Battery'].capacity_in_flow_hours.piecewise_effects_of_investment.plot(
    title='Battery Investment Cost (Economies of Scale)'
)

## Compare Results

In [ ]:
linear_size = fs_linear.solution['Battery|size'].item()
linear_cost = fs_linear.solution['costs'].item()

piecewise_size = fs_piecewise.solution['Battery|size'].item()
piecewise_cost = fs_piecewise.solution['costs'].item()

print('Linear model (0.15 €/kWh constant):')
print(f'  Optimal size: {linear_size:.0f} kWh')
print(f'  Total cost:   {linear_cost:,.1f} €')

print('\nPiecewise model (0.15→0.10→0.05 €/kWh):')
print(f'  Optimal size: {piecewise_size:.0f} kWh')
print(f'  Total cost:   {piecewise_cost:,.1f} €')

if linear_size > 0:
    print(f'\nThe piecewise model builds {(piecewise_size / linear_size - 1) * 100:.0f}% more capacity')
    print(f'at {(1 - piecewise_cost / linear_cost) * 100:.0f}% lower total cost.')
else:
    print(f'\nLinear model builds no battery; piecewise builds {piecewise_size:.0f} kWh.')

**Why the difference?**

- **Linear model**: Each additional kWh costs 0.15€. The optimizer stops when marginal benefit equals marginal cost.
- **Piecewise model**: Additional capacity beyond 300 kWh costs only 0.05€/kWh. The low incremental cost makes the full 600 kWh economically attractive.

## Storage Operation

In [ ]:
fs_piecewise.statistics.plot.balance('Elec')

## Key Concepts

### PiecewiseEffects Structure

```python
fx.PiecewiseEffects(
    # Size breakpoints (what we're sizing)
    piecewise_origin=fx.Piecewise([
        fx.Piece(start=0, end=100),
        fx.Piece(start=100, end=300),
    ]),
    # Cumulative cost breakpoints
    piecewise_shares={
        'costs': fx.Piecewise([
            fx.Piece(start=0, end=15),    # At 100 kWh: total cost is 15€
            fx.Piece(start=15, end=35),   # At 300 kWh: total cost is 35€
        ])
    },
)
```

### Important Rules

1. **Costs are cumulative** - Values represent total cost at each size, not incremental
2. **Segments connect** - End of segment N equals start of segment N+1
3. **Equal segment count** - Origin and shares must have the same number of pieces

## Summary

- **PiecewiseEffects** models non-linear investment costs
- **Economies of scale** make larger systems relatively cheaper
- Cost values are **cumulative totals** at each size breakpoint

### When to Use Which Approach

| Approach | Use When | Example |
|----------|----------|--------|
| Time-varying factors | Efficiency varies with weather | Heat pump COP |
| PiecewiseConversion | Efficiency varies with load | Engine part-load |
| **PiecewiseEffects** | Costs vary with size | Bulk discounts |